In [19]:
import aug_util as aug
import wv_util as wv
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import csv
import tqdm
%matplotlib inline

fdir = '/data/zjc4/'
#Load an image
chip_name = fdir+'train_images/104.tif'
cdir = "/data/zjc4/chipped/train/"
arr = wv.get_image(chip_name)

In [20]:
#Loading our labels
coords1, chips1, classes1 = wv.get_labels(fdir+'xView_train.geojson')

100%|██████████| 601937/601937 [00:04<00:00, 143758.84it/s]


In [21]:
import glob
all_images = glob.glob(fdir+'train_images/*.tif')

In [31]:
import pickle
with open('test_tifs.pkl','rb') as f:
    test_tifs = pickle.load(f)

print(len(all_images))

#Load the class number -> class string label map
labels = {}
with open('xview_class_labels.txt') as f:
    for row in csv.reader(f):
        labels[int(row[0].split(":")[0])] = row[0].split(":")[1]
        pass
    pass

all_images

i = 0
for chip_name in tqdm.tqdm(test_tifs):
    chip_name = fdir+chip_name
    chip_name = (chip_name.split("/")[-1])
    img_name = chip_name.split(".")[0]
    
    coords = coords1[chips1==chip_name]
    classes = classes1[chips1==chip_name].astype(np.int64)
    
    c_img, c_box, c_cls = wv.chip_image(img = arr, coords= coords, 
                                        classes=classes, shape=(600,600))
    # xmin,ymin,xmax,ymax = c_box
    #   0    1   2    3
    for c_idx in (range(c_img.shape[0])):
        # Save the chipped image
        c_name = "{:06}_{:02}".format(int(img_name), c_idx)
        # Save the chipped label
        widths = c_box[c_idx][:,2]-c_box[c_idx][:,0]
        x = c_box[c_idx][:,0]+(widths/2)
        
        heights = c_box[c_idx][:,3]-c_box[c_idx][:,1]
        y = c_box[c_idx][:,1]+(heights/2)
        
        szx = c_img[c_idx].shape[0]
        szy = c_img[c_idx].shape[1]
        nwidths,nheights = widths/szx, heights/szy
        nx,ny = x/szx, y/szy

        nheights = np.round(nheights,6)
        nwidths = np.round(nwidths,6)
        nx,ny = np.round(nx,6),np.round(ny,6)
        
        h_cls = c_cls[c_idx]
        
        classes = [[11,12],[13],[17,18,20,21],\
                   [19,23,24,25,28,29,60,61,65,26],[41,42,50,40,44,45,47,49]]
        matching_cls_idxs = np.copy(np.isin(h_cls,classes[0])) # init class idx
        for idx,c in enumerate(classes):
            idx_filter = np.isin(h_cls,c)
            h_cls[idx_filter] = idx
            matching_cls_idxs = np.copy(np.logical_or(matching_cls_idxs,idx_filter))

        data_labels = np.vstack((h_cls,nx,ny,nwidths,nheights)).T
        # Select only valid labels
        y_valid = np.logical_and(nx-nwidths > 0.1,nx+nwidths < 0.9)
        x_valid = np.logical_and(ny-nheights > 0.1,ny+nheights < 0.9)
        
        valid = np.logical_and(x_valid,np.logical_and(y_valid,matching_cls_idxs))
        
        data_labels = data_labels[valid,:]
        
        # Break if there are no valid labels(o.w. empty label == error)
        if (np.count_nonzero(valid==True) < 1 ): break
        ff_l = "{}labels/{}.txt".format(cdir,c_name)
        np.savetxt(ff_l, data_labels, fmt='%i %1.6f %1.6f %1.6f %1.6f')
        
        img = Image.fromarray(c_img[c_idx])
        ff_i = "{}images/{}.jpg".format(cdir,c_name)
        img.save(ff_i)

  1%|          | 1/170 [00:00<00:27,  6.16it/s]

846


100%|██████████| 170/170 [00:30<00:00,  5.51it/s]


In [32]:
len(lines)

1522

In [33]:
lines = sorted(glob.glob(cdir+"images/*"))
with open(fdir+"chipped/xview_img_valid.txt", mode='w', encoding='utf-8') as myfile:
    myfile.write('\n'.join(lines))

In [34]:
lines[:100]

['/data/zjc4/chipped/train/images/000080_00.jpg',
 '/data/zjc4/chipped/train/images/000091_00.jpg',
 '/data/zjc4/chipped/train/images/000091_01.jpg',
 '/data/zjc4/chipped/train/images/000091_02.jpg',
 '/data/zjc4/chipped/train/images/000091_03.jpg',
 '/data/zjc4/chipped/train/images/000091_04.jpg',
 '/data/zjc4/chipped/train/images/000091_05.jpg',
 '/data/zjc4/chipped/train/images/000091_06.jpg',
 '/data/zjc4/chipped/train/images/000091_07.jpg',
 '/data/zjc4/chipped/train/images/000091_08.jpg',
 '/data/zjc4/chipped/train/images/000091_09.jpg',
 '/data/zjc4/chipped/train/images/000091_10.jpg',
 '/data/zjc4/chipped/train/images/000091_11.jpg',
 '/data/zjc4/chipped/train/images/000091_12.jpg',
 '/data/zjc4/chipped/train/images/000091_13.jpg',
 '/data/zjc4/chipped/train/images/000091_14.jpg',
 '/data/zjc4/chipped/train/images/000091_15.jpg',
 '/data/zjc4/chipped/train/images/000091_16.jpg',
 '/data/zjc4/chipped/train/images/000091_17.jpg',
 '/data/zjc4/chipped/train/images/000094_00.jpg',
